# 2D PDE on regular & irregular domain
## Types of PDEs that the solver supports
This is a solver for linear (constant/variable coefficient) 2D PDEs, i.e., PDEs that can be written in the form of $\mathcal{L}u=f(x, y)$, where $\mathcal{L}$ is a linear combination of $\{\mathit{I}, \frac{\partial}{\partial x}, \frac{\partial}{\partial y}, \frac{\partial^2}{\partial x^2}, \frac{\partial^2}{\partial y^2}, \nabla^2, \dots\}$ whose coefficients __can be either a constant or a function $g(x, y)$__. Additionally, users are free to add any other operators.   
  
Before we begin, let's introduce all the environment variables:

In [1]:
import sys, math, numpy as np
sys.path.append("../src/")
import fdm.solver.fdm_solver as FDM
import util.diff_operator_expression.diff_op_expression as Expr
from util.diff_operators.impl import d2dx, d2dy, laplacian2d
from util.domain_conditions.impl.dirichlet import dirichlet_bc as Dirichlet_BC
from util.domain_conditions.core.domain import domain as Domain

## Regular Domain
Let's introduce the 2D PDE that we're gonna solve:  
$\nabla^2 u = \frac{2}{(1+x)^3} + \frac{2}{(1+y)^3}$ whose boundary conditions are  
$u(x, 0) = 1+\frac{1}{1+x}$ for $x\in [0,1]$,  
$u(x,1)=\frac{1}{2}+\frac{1}{1+x}$ for $x\in[0, 1]$,  
$u(0, y) = 1+\frac{1}{1+y}$ for $y\in[0,1]$,  
$u(1, y)=\frac{1}{2}+\frac{1}{1+y}$ for $y\in[0,1]$.  
Then we define our Dirichlet boundary condition.

In [2]:
domain = Domain.domain(np.array([0, 0]), np.array([1, 1]))
inDomain = lambda x, y: 0 < x < 1 and 0 < y < 1
onBoundary = lambda x, y: abs(x-1) < np.spacing(1) or abs(x) < np.spacing(1) or abs(y-1) < np.spacing(1) or abs(y) < np.spacing(1)
getBoundaryValue = lambda x, y: 1/(1+x) + 1/(1+y) # You can verify that this gives us the boundary conditions

dirichlet = Dirichlet_BC.dirichlet_bc(inDomain, onBoundary, getBoundaryValue, domain) # Boundary condition class

Now we are able to define our solver. Note that we will define the solver on four domains which are discretized in different scales. 

In [3]:
f = lambda x, y: 2/(1+x)**3 + 2/(1+y)**3 # this is the function on the right hand side of our PDE
approximation = []
for n in [9, 19, 39, 79]:
    dx, dy = 1/(n+1), 1/(n+1) # each grid's length
    expression = Expr.diff_operator_expression([laplacian2d.laplacian2d(dx, dy, coefficient=lambda x, y: 1)]) 
    # we only have a laplacian operator in our PDE
    solver = FDM.fdm_solver(expression, f, dirichlet) 
    # Insert the operator expression, function on the right hand side and the boundary condition into our solver
    %timeit -n1 -r1 approximation.append(solver.solve(n, n))
    # n represents grid point numbers. In this case, we have n points in both x and y axis.

31 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
90.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
333 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


We rapidly get the approximated solution to our PDE. Fortunately, we have an analytical solution to this problem, which is $u(x, y) = \frac{1}{1+x}+\frac{1}{1+y}$. We now compare our approximation with the analytical solution using the $L^\infty$ norm.

In [4]:
error = [] # Use this array to record the errors of our approximation.
for index, n in enumerate([9, 19, 39, 79]):
    # We get the approximated solution. Then we compare it with the analytical solution.
    correct_function = lambda x, y: 1/(1+x) + 1/(1+y)
    correct_solution = np.zeros(n**2)
    for j in range(n):
        y = (j+1)/(n+1)
        for i in range(n):
            x = (i+1)/(n+1)
            correct_solution[j*n+i] = correct_function(x, y)
    error.append(max(abs(approximation[index]-correct_solution)))
print(" n", "max_error")
for index, n in enumerate([9, 19, 39, 79]):
    print("{:2d} {:f}".format(n, error[index]))

 n max_error
 9 0.000554
19 0.000140
39 0.000035
79 0.000009


We can see that it converges in second-order of accuracy. This is because that I implemented the second-order of accuracy numerical scheme in the Laplacian operator, which is shown as follows:
```python
class laplacian2d(diff_op.diff_operator):
    def __init__(self, dx, dy, coefficient = 1):
        super().__init__([((0, -1), 1/dy**2), ((-1, 0), 1/dx**2), ((0, 0), -2/dx**2-2/dy**2), ((1, 0), 1/dx**2), ((0, 1), 1/dy**2)], coefficient)
```
The first argument in the super().\_\_init\__ function defines the numerical scheme used by the operator. __Users can modify it at will to increase/decrease its degree of accuracy__.  
  
Here is an explanation of the five point method I used here.
![Five point formula used here](img/five_points_method.png)

## Irregular Domain
This time we consider a 2D PDE on a unit circle domain.  
$\frac{\partial^2u}{\partial x^2}+\frac{\partial^2u}{\partial y^2} = 16(x^2+y^2)$ on $\Omega = \{(x, y):x^2+y^2<1\}$ whose boundary condition is $u(x, y) = 1$ for $(x, y)\in \partial\Omega$.  
Again, I'm going to define the Dirichlet boundary condition.

In [5]:
# These two functions take (x, y) as input, then output its closest point on the unit circle
def get_y_by_x(x, y):
    y1 = math.sqrt(1-x**2)
    y2 = -y1
    if abs(y-y2) < abs(y-y1):
        return (x, y2)
    else:
        return (x, y1)
    
def get_x_by_y(x, y):
    x1 = math.sqrt(1-y**2)
    x2 = -x1
    if abs(x-x2) < abs(x-x1):
        return (x2, y)
    else:
        return (x1, y)

domain = Domain.domain(np.array([-1, -1]), np.array([1, 1]))
inDomain = lambda x, y: x**2 + y**2 < 1
onBoundary = lambda x, y: abs(x**2 + y**2 - 1) < np.spacing(1)
getBoundaryValue = lambda x, y: 1
getNearestPoint = [get_x_by_y, get_y_by_x] # Defined above
dirichlet = Dirichlet_BC.dirichlet_bc(inDomain, onBoundary, getBoundaryValue, domain, getNearestPoint)

Now we are able to define our solver. Note that what I did for the irregular domain is basically identical to what I have done for the regular domain, since the solver would __automatically detect irregular domain and execute the algorithm for irregular domain__. So unlike the other solvers, users of my solver don't have to do any extra work.  
We also have an analytical solution to this problem, which is $u(x, y) = (x^2+y^2)^2$.

In [7]:
f = lambda x, y: 16*(x**2+y**2) # this is the function on the right hand side of our PDE
error = [] # Use this array to record the errors of our approximation.
approximation = []
for index, n in enumerate([9, 19, 39, 79]):
    dx, dy = 1/(n+1), 1/(n+1)
    expression = Expr.diff_operator_expression([d2dx.d2dx(dx, coefficient=lambda x, y: 1), d2dy.d2dy(dy, coefficient=lambda x, y: 1)])
    # we only have d2dx and d2dy in our PDE
    solver = FDM.fdm_solver(expression, f, dirichlet)
    # Insert the operator expression, function on the right hand side and the boundary condition into our solver
    %timeit -r1 -n1 approximation.append(solver.solve(2*n+1, 2*n+1))
    # n represents grid point numbers. In this case, we have n points in both x and y axis.

    
    correct_function = lambda x, y: (x**2+y**2)**2
    correct_solution = np.zeros(solver.vector_len) 
    # solver.vector_len records the number of points we approximated on the irregular domain
    for i in range(solver.vector_len):
        correct_solution[i] = correct_function(*solver._get_coord_by_offset(dx, dy, *solver.index_to_grid[i]))
        # the complex argument gives us the location of the point
    error.append(max(abs(approximation[index]-correct_solution)))
    
print(" n", "max_error")
for index, n in enumerate([9, 19, 39, 79]):
    print("{:2d} {:f}".format(n, error[index]))

106 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
311 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
10.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
 n max_error
 9 0.009951
19 0.002496
39 0.000625
79 0.000156


We can see that a PDE defined on irregular domain takes a longer time to solve. However, it still converges in second-order of accuracy even when the domain is irregular, since I implemented the second-order of accuracy numerical scheme for $\frac{\partial^2u}{\partial x^2}$ and $\frac{\partial^2u}{\partial y^2}$ when the domain is irregular. The following code snipet illustrates how it is implemented:
```python
class d2dx(diff_op.diff_operator):
    def __init__(self, dx, coefficient = 1):
        super().__init__([((-1, 0), 1/dx**2), ((0, 0), -2/dx**2), ((1, 0), 1/dx**2)], coefficient)

    def getIrregularStencil(self, dx, tau, direction):
        # direction = 1 represents positive y axis; -1 represents negative y axis
        c1, c2, c3, c4 = (tau-1)/(tau+2), 2*(2-tau)/(tau+1), (tau-3)/tau, 6/(tau*(tau+1)*(tau+2))
        if direction == diff_op.Direction.POSITIVE:
            return [irregular_stencil_node((-2, 0), c1/dx**2), irregular_stencil_node((-1, 0), c2/dx**2), irregular_stencil_node((0, 0), c3/dx**2), irregular_stencil_node((tau, 0), c4/dx**2, True)]
        elif direction == diff_op.Direction.NEGATIVE:
            return [irregular_stencil_node((-tau, 0), c4/dx**2, True), irregular_stencil_node((0, 0), c3/dx**2), irregular_stencil_node((1, 0), c2/dx**2), irregular_stencil_node((2, 0), c1/dx**2)]
```
When an irregular domain is detected by my solver, it would automatically call the function getIrregularStencil() to get the stencil for irregular domains.